In [2]:
import urllib2
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt, fabs, exp
from sklearn import linear_model
from sklearn.linear_model import enet_path
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
target_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
data = urllib2.urlopen(target_url)

xList  = []
labels = []

for line in data:
    row = line.strip().split(",")
    xList.append(row)
    
xNum = []

for row in xList:
    lastcol = row.pop()
    if lastcol == "M":
        labels.append(1.0)
    else:
        labels.append(0.0)
        
    attRow = [float(elt) for elt in row]
    xNum.append(attRow)
    
print (xList)

print ("bhavydfabdajfabdfhjakbhjlabfhak")
print (xNum)

# number of rows and columns
nrow = len(xNum)
ncol = len(xNum[0])

alpha = 1.0

# Calculate mean and variance
xMeans = []
xSD = []

for i in range(ncol):
    mean = sum([xNum[j][i] for j in range(nrow)])/nrow
    xMeans.append(mean)
    coldiff = [(xNum[j][i] - mean) for j in range(nrow)]
    sumSq = sum([coldiff[i] * coldiff[i] for i in range(nrow)])
    stdDev = sqrt(sumSq/nrow)
    xSD.append(stdDev)
    
# Normalize xNum
xNormalized = []
for i in range(nrow):
    rowNormalized = [(xNum[i][j] - xMeans[j])/xSD[j] for j in range(ncol)]
    xNormalized.append(rowNormalized)
    
# Normalize labels to center

meanLabel = sum(labels)/nrow
sdLabel = sqrt(sum([(labels[i] - meanLabel) * (labels[i] - meanLabel) for i in range(nrow)])/nrow)

labelNormalized = [(labels[i] - meanLabel)/sdLabel for i in range(nrow)]

# Number of cross validation folds
nxval = 10

for ixval in range(nxval):
    # Define test and training sets
    idxTest = [a for a  in range(row) if a%nxval == ixval%nxval]
    idxTrain = [a for a in range(row) if a%nxval != ixval%nxval]
    
    # Define test and training attribute and label sets
    xTrain = np.array(xNormalized[r] for r in idxTrain)
    xTest = np.array(xNormalized[r] for r in idxTest)
    labelTrain = np.array(labelNormalized[r] for r in idxTrain)
    labelTest = np.array(labelNormalized[r] for r in idxTest)
    
    alphas, coefs, _ = enet_path(xTrain, labelTrain,ll_ratio=0.8, fit_intercept=False, return_models=False)
    
    # Apply coefficients to test data to produce predictions and accumulate 
    if xval == 0:
        pred = np.dot(xTest, coefs)
        yOut = labelTest
        
    else:
        # Accumulate predictions
        predTemp = np.array(pred)
        pred = np.concatenate((predTemp, np.dot(xTest, coefs)), axis=0)
        
    
# Calculate misclassification error
misClassRate = []
_, nPred = pred.shape
for iPred in range(1, nPred):
    predList = list(pred[:, iPred])
    errCnt = 0.0
    for irow in range(nrow):
        if (predList[irow] < 0.0) and (yOut[irow] >= 0.0):
            errCnt += 1.0
        elif (predList[irow] >= 0.0) and (yOut[irow] < 0.0):
            errCnt += 1.0
            
    missClassRate.append(errCnt/nrow)
    
# Find minimum point for plot and for print
minError = min(misClassRate)
idxMin = missClassRate.index(minError)
plotAlphas = list(alphas[1: len(alphas)])

plt.figure()
plt.plot(plotAlphas, midClassRate, label="Misclassification error across folds", linewidth=4)
plt.axvline(plotAlphas[idxMin], linestyle='--', label="CV estimate for best alpha")
plt.legend()
plt.semilogx()
ax = plt.gca()
ax.invert_xaxis()
plt.ylabel("Misclassification error")
plt.xlabel("Alpha")
plt.axis("tight")

plt.show()

# Calculate AUC

idxPos = [i for i in range(nrow) if yOut[i] > 0.0]
yOutBin = [0] * nrow
for i in idxPos:
    yOutBin[i] = 1
    
auc = []
for iPred in range(1, nPred):
    predList = list(pred[:, iPred])
    aucCalc = roc_auc_score(yOutBin, predList)
    auc.append(aucCalc)
    
maxAUC = max(auc)
idxMax = auc.index(maxAuc)

plt.figure()
plt.plot(plotAlphas, auc, label="AUC across folds", linewidth=4)
plt.axvline(plotAlphas[idxMax], linestyle='--', label="CV estimate for best alpha")
plt.legend()
plt.semilogx()
ax = plt.gca()
ax.invert_xaxis()
plt.ylabel("Area under the ROC curve")
plt.xlabel("Alpha")
plt.axis("tight")
plt.show()

# Plot best version of ROC curve
fpr, tpr, thresh = roc_curve(yOutBin, list(pred[:, idxMax]))

ctClass = [i*0.01 for i in range(101)]

plt.plot(fpr, tpr, linewidth=4)
plt.plot(ctClass, ctClass, linestyle=':')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

print ("Best value of Misclassification error ", misClassRate[idxMin])
print ("Best alpha for Misclassfication error ", plotAlphas[idxMin])
print ("")
print ("Confusion matrices for different threshold values")

# Pick some points along the curve to print. There are 208 points, the extremes are not useful

# Sample at 52, 104, 156. Use the calculated values of tpr, fpr along with definitions and threshold

# P = +ive cases
P = len(idxPos)

# N = -ive cases
N = nrow - P

# TP = True +ive = tpr *P
TP = tpr[52] * P

#FN = False -ive = P - TP
FN = P - TP

# FP = False +ive = fpr * N
FP = fpr[52] * N

# TN = True -ive = N - FP
TN = N - FP

print ("Threshold Value =  ", thresh[52])
print ("TP = ", TP, "FP = ", FP)
print ("FN = ", FN, "TN = ", TN)

TP = tpr[104] * P; FN = P - TP; FP = fpr[104] * N; TN = N - FP

print ("Threshold Value =  ", thresh[104])
print ("TP = ", TP, "FP = ", FP)
print ("FN = ", FN, "TN = ", TN)

TP = tpr[156] * P; FN = P - TP; FP = fpr[156] * N; TN = N - FP

print ("Threshold Value =  ", thresh[156])
print ("TP = ", TP, "FP = ", FP)
print ("FN = ", FN, "TN = ", TN)


